In [1]:
import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from datetime import datetime
import tensorflow as tf

/opt/homebrew/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32719/3878954045.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [8]:
#clean up data for
data = pd.read_csv('full_data.csv')
data['Date'] = pd.to_datetime(data['Date'])

btc = data.loc[data['Crypto']=='BTC']
btc = btc.sort_values(by='Date')
btc = btc.reset_index(drop=True)
btc = btc.drop(['Crypto'],axis=1)
btc = btc.set_index('Date')

eth = data.loc[data['Crypto']=='ETH']
eth = eth.sort_values(by='Date')
eth = eth.reset_index(drop=True)
eth = eth.drop(['Crypto'],axis=1)
eth = eth.set_index('Date')

ltc = data.loc[data['Crypto']=='LTC']
ltc = ltc.sort_values(by='Date')
ltc = ltc.reset_index(drop=True)
ltc = ltc.drop(['Crypto'],axis=1)
ltc = ltc.set_index('Date')

#train for optimization only till then
btc = btc.loc[btc.index>=datetime(2022,5,31)]
eth = eth.loc[eth.index>=datetime(2022,5,31)]
ltc = ltc.loc[ltc.index>=datetime(2022,5,31)]

In [3]:
from arch.unitroot import ADF
from arch.unitroot import KPSS
from arch.unitroot import PhillipsPerron
from sklearn.linear_model import LinearRegression

def strat_creation():
  # Create a Stratey
  class spread_strat(bt.Strategy):

      params = (
          ('history', 60),
        )

      def log(self, txt, dt=None):
          ''' Logging function for this strategy '''
          if dt is None:
              # Access the datetime index from the current line in the data series
              dt = self.datas[0].datetime.datetime(0)

          # Check if dt is still a float (the internal representation for Backtrader), and convert it if needed
          if isinstance(dt, float):
              # Convert backtrader float date to datetime
              dt = bt.num2date(dt)

          # Format datetime object to string
          dt_str = dt.strftime('%Y-%m-%d %H:%M:%S')
          print('%s, %s' % (dt_str, txt))

      def close_all_positions(self):
        count = 0
        names = ['BTC', 'LTC', 'ETH']
        for data in self.datas:
          position = self.getposition(data)
          if position.size != 0:
              action = 'SELL' if position.size > 0 else 'BUY'
              self.log_trade(action, -1*position.size, data.close[0], names[count])
              self.close(data)
          count+=1
        return

      def log_trade(self, action, size, price, asset):
        self.trade_logs.append({
                'Date': self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H:%M:%S'),
                'Action': action,
                'Size': size,
                'Price': price,
                'Asset': asset
            })
        return

      def __init__(self):
        self.data0 = self.datas[0].close #btc
        self.data1 = self.datas[1].close #eth
        self.data2 = self.datas[2].close #ltc

        #constant for strat
        self.cointegrated = False
        self.model_built = False
        self.spread_val = None
        self.upper_bound = None
        self.lower_bound = None
        self.mean_ = None
        self.sigma = 1.5
        self.coefficients = None
        self.order = None
        self.position_opened = False
        self.position_opened_time = None
        self.stop_loss_spread_val = None
        self.stop_loss_constant = 0.10
        self.position_type = None
        self.model_date = self.datas[0].datetime.datetime(0)
        self.trade_logs = []

      def clear_trade_vars(self):
        self.cointegrated = False
        self.model_built = False
        self.spread_val = None
        self.upper_bound = None
        self.lower_bound = None
        self.mean_ = None
        self.coefficients = None
        self.order = None
        self.position_opened = False
        self.position_opened_time = None
        self.stop_loss_spread_val = None
        self.position_type = None
        #self.model_date = self.datas[0].datetime.datetime(0)
        return

      def next(self):
        current_datetime = self.datas[0].datetime.datetime(0)
        current_hour = current_datetime.hour
        current_minute = current_datetime.minute
        series1 = pd.Series(self.data0.get(size=self.params.history))
        series2 = pd.Series(self.data1.get(size=self.params.history))
        series3 = pd.Series(self.data2.get(size=self.params.history))
        series1_diff = series1.diff().dropna()  # taking the diff of both series
        series2_diff = series2.diff().dropna()
        series3_diff = series3.diff().dropna()
        tickers = [series1_diff, series2_diff, series3_diff]

        if len(self.data0) >= self.params.history: #check if there is sufficient data for looking back
          if self.model_built == False:
            ##we build the cointegrated model
            for i in range(len(tickers)): #iterate through series and check for stationarity
              adf = ADF(tickers[i])
              pp = PhillipsPerron(tickers[i])
              kpss = KPSS(tickers[i])
              #print(f'adf: {adf.pvalue}, pp: {pp.pvalue}, kpss:{kpss.pvalue}')
              if adf.pvalue < 0.05 and pp.pvalue < 0.05 and kpss.pvalue > 0.10: #check weather pass all the tests
                self.cointegrated = True
              else:
                self.cointegrated = False

          if self.cointegrated == True and self.model_built == False:
            ##we create the model now
            lin_model = LinearRegression()
            X = np.column_stack((series2.values, series3.values)) #ETH and LTC values
            lin_model.fit(X, series1.values)
            self.coefficients = [1] + list(-1*lin_model.coef_)
            spread = series1 + self.coefficients[1]*series2 + self.coefficients[2]*series3
            self.mean_ = np.mean(spread)
            self.upper_bound = self.mean_ + self.sigma*np.std(spread)
            self.lower_bound = self.mean_ - self.sigma*np.std(spread)
            self.model_built = True
            self.model_date = current_datetime
            self.spread_val = self.data0[0] + self.coefficients[1]*self.data1[0] + self.coefficients[2]*self.data2[0]
            self.log(f'Lower bound {self.lower_bound}, Upper bound {self.upper_bound}, spread_val: {self.spread_val}, coefs:{self.coefficients}')

          if self.model_built == True:
            self.spread_val = self.data0[0] + self.coefficients[1]*self.data1[0] + self.coefficients[2]*self.data2[0]
            #self.log(f'spread val: {spread_val}')
            self.log(f'spread value: {self.spread_val} btc:{self.data0[0]}, eth: {self.data1[0]}, ltc: {self.data2[0]}')
            if not self.position: #NOT CURRENTLY IN A POSITION
              #we go LONG the spread
              if self.spread_val < self.lower_bound:
                self.log(f'BUY CREATED, spread val: {self.spread_val}')
                self.buy(self.datas[0], size = 1) #buy BTC
                self.log_trade('BUY', 1, self.datas[0][0], 'BTC')
                self.position_opened = True
                self.position_opened_time = current_datetime
                for i in range(1, len(self.coefficients)):
                  if self.coefficients[i] > 0:
                    self.buy(self.datas[i], size=round(self.coefficients[i],2))
                    self.log_trade('BUY', self.coefficients[i], self.datas[i][0], 'ETH' if i == 1 else 'LTC')
                  if self.coefficients[i] < 0:
                    self.sell(self.datas[i], size=round(self.coefficients[i],2))
                    self.log_trade('SELL', self.coefficients[i], self.datas[i][0], 'ETH' if i == 1 else 'LTC')
                self.stop_loss_spread_val = self.spread_val - self.spread_val * self.stop_loss_constant if self.spread_val > 0 else self.spread_val + self.spread_val * self.stop_loss_constant
                self.position_type = "LONG"
              #we go SHORT the spread
              if self.spread_val > self.upper_bound:
                self.log(f'SELL CREATED, spread val: {self.spread_val}')
                self.sell(self.datas[0], size = 1) #sell BTC
                self.log_trade('SELL', -1, self.datas[0][0], 'BTC')
                self.position_opened = True
                self.position_opened_time = current_datetime
                for i in range(1, len(self.coefficients)):
                  if self.coefficients[i] > 0: #need to do the opposite to short the spread
                    self.sell(self.datas[i], size= round(self.coefficients[i],2))
                    self.log_trade('SELL', -1*self.coefficients[i], self.datas[i][0], 'ETH' if i == 1 else 'LTC')
                  if self.coefficients[i] < 0:
                    self.buy(self.datas[i], size = round(self.coefficients[i],2))
                    self.log_trade('BUY', -1*self.coefficients[i], self.datas[i][0], 'ETH' if i == 1 else 'LTC')
                self.stop_loss_spread_val = self.spread_val + self.spread_val * self.stop_loss_constant if self.spread_val > 0 else self.spread_val - self.spread_val * self.stop_loss_constant
                self.position_type = "SHORT"

            elif self.position: #IN A POSITION check to take profits and close out existing trade
              position_size = self.position.size #check the first asset BTC
              #close out our LONG position
              if position_size > 0 and self.spread_val > self.upper_bound:
                self.log(f'CLOSE EXISTING long position: {self.spread_val}')
                self.close_all_positions()
                self.clear_trade_vars()
              #close out SHORT position
              elif position_size < 0 and self.spread_val < self.lower_bound:
                self.log(f'CLOSE EXISTING short positon: {self.spread_val}')
                self.close_all_positions()
                self.clear_trade_vars()
              #check stop loss
              elif self.position:
                self.check_stop_loss()

          if self.model_built == True and self.position_opened == False: #only rebuild if not in a position
            if abs(self.spread_val) > abs(1.50*self.mean_) or (current_datetime - self.model_date).days >= 30: #if the current spread is to far away from out cointegrated model we rebuild the model
              print('REBUILD THE MODEL')
              self.clear_trade_vars()
          days_close = 90
          if self.position_opened and (current_datetime - self.position_opened_time).days >= days_close:
            self.close_all_positions()
            self.clear_trade_vars()
            self.log(f'POSITION CLOSED after {days_close} days at {current_datetime}')


      def check_stop_loss(self):
        if self.spread_val < self.stop_loss_spread_val and self.position_type == "LONG":
          self.log(f'Long position stopped out: {self.spread_val}')
          self.close_all_positions()
          self.clear_trade_vars()
        elif self.spread_val > self.stop_loss_spread_val and self.position_type == "SHORT":
          self.log(f'Short position stopped out: {self.spread_val}')
          self.close_all_positions()
          print('in stop')
          self.clear_trade_vars()

        return

      def stop(self):
        # Close all positions at the end of the strategy
        self.log('Closing all positions at the end of the backtest')
        self.close_all_positions()
        self.trade_logs_df = pd.DataFrame(self.trade_logs)
        self.trade_logs_df['value'] = self.trade_logs_df['Size']*self.trade_logs_df['Price']
        print(self.trade_logs_df)

  return spread_strat

In [5]:
cerebro = bt.Cerebro()
TestStrategy = strat_creation()
cerebro.addstrategy(TestStrategy)
btc_bt = bt.feeds.PandasData(dataname=btc)
cerebro.adddata(btc_bt)
eth_bt = bt.feeds.PandasData(dataname=eth)
cerebro.adddata(eth_bt)
ltc_bt = bt.feeds.PandasData(dataname=ltc)
cerebro.adddata(ltc_bt)
initial_cash = 1000000
cerebro.broker.set_cash(initial_cash)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio", riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
cerebro.broker.setcommission(commission=0.0)

results = cerebro.run()
strategy = results[0]

metrics = {
        'method': [],
        'Total Net Profit': [],
        'Gross Profit': [],
        'Gross Loss': [],
        'Percent Profitable': [],
        'Winning Trades': [],
        'Losing Trades': [],
        'Avg. Trade Net Profit': [],
        'Avg. Winning Trade': [],
        'Avg. Losing Trade': [],
        'Ratio Avg. Win:Avg. Loss': [],
        'Largest Winning Trade': [],
        'Largest Losing Trade': [],
        'Max. Consecutive Winning Trades': [],
        'Max. Consecutive Losing Trades': [],
        'Avg. Bars in Total Trades': [],
        'Avg. Bars in Winning Trades': [],
        'Avg. Bars in Losing Trades': [],
        'Max. Drawdown': []
    }

# Extract metrics from analyzers
trade_analyzer = strategy.analyzers.trade_analyzer.get_analysis()
sharpe_ratio = strategy.analyzers.sharpe_ratio.get_analysis()
drawdown = strategy.analyzers.drawdown.get_analysis()
returns = strategy.analyzers.returns.get_analysis()
# Calculate custom metrics
total_net_profit = trade_analyzer.won.pnl.total + trade_analyzer.lost.pnl.total
total_gross_profit = trade_analyzer.won.pnl.total
total_gross_loss = trade_analyzer.lost.pnl.total
total_number_trades = trade_analyzer.total.total
percent_profitable = (trade_analyzer.won.total / trade_analyzer.total.total) * 100
winning_trades = trade_analyzer.won.total
losing_trades = trade_analyzer.lost.total
avg_trade_net_profit = total_net_profit / trade_analyzer.total.total
avg_winning_trade = trade_analyzer.won.pnl.average
avg_losing_trade = trade_analyzer.lost.pnl.average
ratio_avg_win_loss = avg_winning_trade/avg_losing_trade
largest_winning_trade = trade_analyzer.won.pnl.max
largest_losing_trade = trade_analyzer.lost.pnl.max
max_consecutive_winning_trades = trade_analyzer.streak.won.longest
max_consecutive_losing_trades = trade_analyzer.streak.lost.longest
avg_bars_in_total_trades = trade_analyzer.len.total
avg_bars_in_winning_trades = trade_analyzer.len.won.total
avg_bars_in_losing_trades = trade_analyzer.len.lost.total
max_drawdown = drawdown.max.moneydown

metrics['method'].append('Cointegration')
metrics['Total Net Profit'].append(total_net_profit)
metrics['Gross Profit'].append(total_gross_profit)
metrics['Gross Loss'].append(total_gross_loss)
metrics['Percent Profitable'].append(percent_profitable)
metrics['Winning Trades'].append(winning_trades)
metrics['Losing Trades'].append(losing_trades)
metrics['Avg. Trade Net Profit'].append(avg_trade_net_profit)
metrics['Avg. Winning Trade'].append(avg_winning_trade)
metrics['Avg. Losing Trade'].append(avg_losing_trade)
metrics['Ratio Avg. Win:Avg. Loss'].append(ratio_avg_win_loss)
metrics['Largest Winning Trade'].append(largest_winning_trade)
metrics['Largest Losing Trade'].append(largest_losing_trade)
metrics['Max. Consecutive Winning Trades'].append(max_consecutive_winning_trades)
metrics['Max. Consecutive Losing Trades'].append(max_consecutive_losing_trades)
metrics['Avg. Bars in Total Trades'].append(avg_bars_in_total_trades)
metrics['Avg. Bars in Winning Trades'].append(avg_bars_in_winning_trades)
metrics['Avg. Bars in Losing Trades'].append(avg_bars_in_losing_trades)
metrics['Max. Drawdown'].append(max_drawdown)

pd.DataFrame.from_dict(metrics).T

/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32719/2132275407.py:94: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series1 = pd.Series(self.data0.get(size=self.params.history))
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32719/2132275407.py:95: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series2 = pd.Series(self.data1.get(size=self.params.history))
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32719/2132275407.py:96: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series3 = pd.Series(self.data2.get(size=self.params.history))


2022-07-29 00:00:00, Lower bound 6615.777335854615, Upper bound 11618.936291163827, spread_val: 5546.1665065832185, coefs:[1, -14.109425333213848, 98.08951040138608]
2022-07-29 00:00:00, spread value: 5546.1665065832185 btc:23952.0, eth: 1733.17, ltc: 61.66
2022-07-29 00:00:00, BUY CREATED, spread val: 5546.1665065832185
2022-07-30 00:00:00, spread value: 6012.177411590415 btc:23979.0, eth: 1702.82, ltc: 61.77
2022-07-31 00:00:00, spread value: 5618.75481515855 btc:23804.0, eth: 1721.29, ltc: 62.2
2022-08-01 00:00:00, spread value: 5901.6567376861685 btc:23027.0, eth: 1622.95, ltc: 58.86
2022-08-02 00:00:00, spread value: 5486.107842950522 btc:23015.0, eth: 1650.3, ltc: 58.68
2022-08-03 00:00:00, spread value: 5990.419022369261 btc:23324.0, eth: 1641.88, ltc: 59.46
2022-08-04 00:00:00, spread value: 5879.211057330965 btc:22503.0, eth: 1590.67, ltc: 59.33
2022-08-05 00:00:00, spread value: 5281.0018777870555 btc:22969.0, eth: 1679.93, ltc: 61.32
2022-08-06 00:00:00, spread value: 5002.8

,0
method,Cointegration
Total Net Profit,-4854.538
Gross Profit,18665.0875
Gross Loss,-23519.6255
Percent Profitable,47.058824
Winning Trades,24
Losing Trades,24
Avg. Trade Net Profit,-95.18702
Avg. Winning Trade,777.711979
Avg. Losing Trade,-979.984396


In [11]:
total_net_profit = cerebro.broker.getvalue() - initial_cash
print('total net profit',total_net_profit)
print('broker value',cerebro.broker.getvalue())

sum p/l -4854.538000000011
total net profit -3936.726700000232
broker value 996063.2732999998


In [7]:
df_coint = pd.DataFrame.from_dict(metrics).T
df_coint.to_csv('final_writeup/coint.csv')